
<div style="text-align: center; line-height: 0; padding-top: 9px;">
  <img src="https://databricks.com/wp-content/uploads/2018/03/db-academy-rgb-1200px.png" alt="Databricks Learning">
</div>



# LAB - Batch Deployment

Welcome to the "Batch Deployment" lab! This lab focuses on batch deployment of machine learning models using Databricks. you will engage in tasks related to model inference, model registry, and explore performance results for feature such as Liquid Clustering using `CLUSTER BY`.

**Learning Objectives:**

By the end of this lab, you will be able to;

+ **Task 1: Load Dataset**
    + Load Dataset
    + Split the dataset to features and response sets

+ **Task 2: Inference with feature table**

    + Create Feature Table
    + Setup Feature Lookups
    + Fit and Register a Model with UC using Feature Table
    + Perform batch inference using Feature Engineering's  **`score_batch`** method.

+ **Task 3: Assess Liquid Clustering:**

    + Evaluate the performance results for specific optimization techniques:
        + Liquid Clustering


## Requirements

Please review the following requirements before starting the lesson:

* To run this notebook, you need to use one of the following Databricks runtime(s): **15.4.x-cpu-ml-scala2.12**


## Classroom Setup

Before starting the Lab, run the provided classroom setup script. This script will define configuration variables necessary for the demo. Execute the following cell:

In [0]:
%run ../Includes/Classroom-Setup-02Lab

Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


Resetting the learning environment:
| dropping the catalog "labuser8891327_1736105957_wp0t_da"...(0 seconds)

Skipping install of existing datasets to "dbfs:/mnt/dbacademy-datasets/machine-learning-model-deployment/v01"

Validating the locally installed datasets:
| listing local files...(0 seconds)
| validation completed...(0 seconds total)
Creating & using the catalog "labuser8891327_1736105957_wp0t_da"...(1 seconds)

Predefined tables in "labuser8891327_1736105957_wp0t_da.default":
| -none-

Predefined paths variables:
| DA.paths.working_dir: dbfs:/mnt/dbacademy-users/labuser8891327_1736105957@vocareum.com/machine-learning-model-deployment
| DA.paths.datasets:    dbfs:/mnt/dbacademy-datasets/machine-learning-model-deployment/v01

Setup completed (5 seconds)


**Other Conventions:**

Throughout this Lab, we'll refer to the object `DA`. This object, provided by Databricks Academy, contains variables such as your username, catalog name, schema name, working directory, and dataset locations. Run the code block below to view these details:

In [0]:
print(f"Username:          {DA.username}")
print(f"Catalog Name:      {DA.catalog_name}")
print(f"Schema Name:       {DA.schema_name}")
print(f"Working Directory: {DA.paths.working_dir}")
print(f"User DB Location:  {DA.paths.datasets}")

Username:          labuser8891327_1736105957@vocareum.com
Catalog Name:      labuser8891327_1736105957_wp0t_da
Schema Name:       default
Working Directory: dbfs:/mnt/dbacademy-users/labuser8891327_1736105957@vocareum.com/machine-learning-model-deployment
User DB Location:  dbfs:/mnt/dbacademy-datasets/machine-learning-model-deployment/v01


## Task 1: Load Dataset

+ Load a dataset:
  + Define the dataset path
  + Define the primary key (`customerID`), response variable (`Churn`), and feature variables (`SeniorCitizen`, `tenure`, `MonthlyCharges`, `TotalCharges`) for further processing.
  + Read the dataset, casting relevant columns to the correct data types, and drop any rows with missing values
+ Split the dataset into training and testing sets
  + Separate the features and the response for the training set


In [0]:
from pyspark.sql.functions import col

# dataset path
dataset_p_telco = f"{DA.paths.datasets}/telco/telco-customer-churn.csv"

# features to use
primary_key = "customerID"
response = "Churn"
features = ["SeniorCitizen", "tenure", "MonthlyCharges", "TotalCharges"]

# Read dataset (and drop nan)
telco_df = spark.read.csv(dataset_p_telco, inferSchema=True, header=True, multiLine=True, escape='"')\
            .withColumn("TotalCharges", col("TotalCharges").cast('double'))\
            .withColumn("SeniorCitizen", col("SeniorCitizen").cast('double'))\
            .withColumn("Tenure", col("tenure").cast('double'))\
            .na.drop(how='any')

# Split with 80 percent of the data in train_df and 20 percent of the data in test_df
train_df, test_df = telco_df.randomSplit([.8, .2], seed=42)

# Separate features and ground-truth
features_df = train_df.select(primary_key, *features)
response_df = train_df.select(primary_key, response)

##Task 2: Inference with feature table
In this task, you will perform batch inference using a feature table. Follow the steps below:

+ **Step 1: Create Feature Table**

+ **Step 2: Setup Feature Lookups**

+ **Step 3: Fit and Register a Model with UC using Feature Table**

+ **Step 4: Use the Model for Inference**


**Step 1: Create Feature Table**
  + Begin by creating a feature table that incorporates the relevant features for inference. This involves selecting the appropriate columns, performing any necessary transformations, and storing the resulting data in a feature table.

In [0]:
from databricks.feature_engineering import FeatureEngineeringClient

# Prepare feature set
features_df_all = telco_df.select(primary_key, *features)

# Feature table definition
fe = FeatureEngineeringClient()
feature_table_name = f"{DA.catalog_name}.{DA.schema_name}.features"

# Drop table if exists
try:
    fe.drop_table(name=feature_table_name)
except:
    pass

# Create feature table
fe.create_table(
    name=feature_table_name,
    df=features_df_all,
    primary_keys=[primary_key],
    description="Lab feature table"
)

2025/01/05 20:53:57 INFO databricks.ml_features._compute_client._compute_client: Setting columns ['customerID'] of table 'labuser8891327_1736105957_wp0t_da.default.features' to NOT NULL.
2025/01/05 20:53:59 INFO databricks.ml_features._compute_client._compute_client: Setting Primary Keys constraint ['customerID'] on table 'labuser8891327_1736105957_wp0t_da.default.features'.
2025/01/05 20:54:08 INFO databricks.ml_features._compute_client._compute_client: Created feature table 'labuser8891327_1736105957_wp0t_da.default.features'.


<FeatureTable: name='labuser8891327_1736105957_wp0t_da.default.features', table_id='49d92a97-9fdb-4bcd-bc31-f11b7b0ccaff', description='Lab feature table', primary_keys=['customerID'], partition_columns=[], features=['customerID', 'SeniorCitizen', 'tenure', 'MonthlyCharges', 'TotalCharges'], creation_timestamp=1736110437198, online_stores=[], notebook_producers=[], job_producers=[], table_data_sources=[], path_data_sources=[], custom_data_sources=[], timestamp_keys=[], tags={}>

**Step 2: Setup Feature Lookups**
  + Set up a feature lookup to create a training set from the feature table. 
  + Specify the `lookup_key` based on the columns that uniquely identify records in your feature table.

In [0]:
from databricks.feature_engineering import FeatureLookup

fl_handle = FeatureLookup(
    table_name=feature_table_name,
    lookup_key=[primary_key]
)

#  Create a training set based on feature lookup
training_set_spec = fe.create_training_set(
    df=response_df,
    label=response,
    feature_lookups=[fl_handle],
    exclude_columns=[primary_key]
)

# Load training dataframe based on defined feature-lookup specification
training_df = training_set_spec.load_df()

**Step 3: Fit and Register a Model with UC using Feature Table**
  + Fit and register a Machine Learning Model using the created training set.
  + Train a model on the training set and register it in the model registry.

In [0]:
import mlflow
import warnings
from mlflow.types.utils import _infer_schema

# Point to UC model registry
mlflow.set_registry_uri("databricks-uc")
client = mlflow.MlflowClient()

# helper function that we will use for getting latest version of a model
def get_latest_model_version(model_name):
    """Helper function to get latest model version"""
    model_version_infos = client.search_model_versions("name = '%s'" % model_name)
    return max([model_version_info.version for model_version_info in model_version_infos])

# Train a sklearn Decision Tree Classification model
from sklearn.tree import DecisionTreeClassifier

# Convert data to pandas dataframes
X_train_pdf = training_df.drop(primary_key, response).toPandas()
Y_train_pdf = training_df.select(response).toPandas()
clf = DecisionTreeClassifier(max_depth=3, random_state=42)

# End the active MLflow run before starting a new one
mlflow.end_run()

with mlflow.start_run(run_name="Model-Batch-Deployment-lab-With-FS") as mlflow_run:

    # Enable automatic logging of input samples, metrics, parameters, and models
    mlflow.sklearn.autolog(
        log_input_examples=True,
        log_models=False,
        log_post_training_metrics=True,
        silent=True)
    
    clf.fit(X_train_pdf, Y_train_pdf)

    # Infer output schema
    try:
        output_schema = _infer_schema(Y_train_pdf)
    except Exception as e:
        warnings.warn(f"Could not infer model output schema: {e}")
        output_schema = None

    model_name = f"{DA.catalog_name}.{DA.schema_name}.ml_model"
    
    # Log using feature engineering client and push to registry
    fe.log_model(
        model=clf,
        artifact_path="decision_tree",
        flavor=mlflow.sklearn,
        training_set=training_set_spec,
        output_schema=output_schema,
        registered_model_name= model_name
    )

    # Set model alias (i.e. Champion)
    client.set_registered_model_alias(model_name, "Champion", get_latest_model_version(model_name))

Uploading artifacts:   0%|          | 0/1 [00:00<?, ?it/s]

/databricks/python/lib/python3.11/site-packages/mlflow/types/schema.py:636: FutureWarning: `optional` is deprecated and will be removed in a future version of MLflow. Use `required` instead.
  warnings.warn(
/databricks/python/lib/python3.11/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


Uploading artifacts:   0%|          | 0/14 [00:00<?, ?it/s]

Successfully registered model 'labuser8891327_1736105957_wp0t_da.default.ml_model'.


Uploading artifacts:   0%|          | 0/14 [00:00<?, ?it/s]

Created version '1' of model 'labuser8891327_1736105957_wp0t_da.default.ml_model'.
2025/01/05 20:55:27 INFO mlflow.tracking._tracking_service.client: 🏃 View run Model-Batch-Deployment-lab-With-FS at: dbc-7a7a1ea4-7ba2.cloud.databricks.com/ml/experiments/3489702795208298/runs/67a5e450833f43aebb2abd78fab1fb95.
2025/01/05 20:55:27 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: dbc-7a7a1ea4-7ba2.cloud.databricks.com/ml/experiments/3489702795208298.


**Step 4: Use the Model for Inference**
  + Utilize the feature engineering client's `score_batch()` method for inference.
  + Provide the model URI and a dataframe containing primary key information for the inference.

In [0]:
# Load the model
model_uri = f"models:/{model_name}@champion"
# Define the test dataset
test_features_df = test_df.select("customerID")

# Perform batch inference using Feature Engineering's score_batch method
result_df = fe.score_batch(
    model_uri=model_uri,
    df=test_features_df,
    result_type='string'  # Update with the desired result type
)

# Display the inference results
display(result_df)

2025/01/05 20:55:31 WARNING mlflow.pyfunc: Calling `spark_udf()` with `env_manager="local"` does not recreate the same environment that was used during training, which may lead to errors or inaccurate predictions. We recommend specifying `env_manager="conda"`, which automatically recreates the environment that was used to train the model and performs inference in the recreated environment.


2025/01/05 20:55:32 INFO mlflow.models.flavor_backend_registry: Selected backend for flavor 'python_function'


customerID,SeniorCitizen,tenure,MonthlyCharges,TotalCharges,prediction
0004-TLHLJ,0.0,4.0,73.9,280.85,Yes
0013-SMEOE,1.0,71.0,109.7,7904.25,No
0015-UOCOJ,1.0,7.0,48.2,340.35,No
0019-EFAEP,0.0,72.0,101.3,7261.25,No
0023-HGHWL,1.0,1.0,25.1,25.1,No
0030-FNXPP,0.0,3.0,19.85,57.2,No
0042-RLHYP,0.0,69.0,19.7,1396.9,No
0057-QBUQH,0.0,43.0,25.1,1070.15,No
0078-XZMHT,0.0,72.0,85.15,6316.2,No
0080-EMYVY,0.0,14.0,51.45,727.85,No


## Task 3: Assess Liquid Clustering:

Evaluate the performance results for specific optimization techniques, such as: Liquid Clustering Follow the step-wise instructions below:  
+ **Step 1:** Create `batch_inference_liquid_clustering` table and import the following columns: `customerID`, `Churn`, `SeniorCitizen`, `tenure`, `MonthlyCharges`, `TotalCharges`, and `prediction`.
+ **Step 2:**  Begin by assessing Liquid Clustering, an optimization technique for improving performance by physically organizing data based on a specified clustering column.
+ **Step 3:**  Optimize the target table for Liquid Clustering.
+ **Step 4:** Specify the `CLUSTER BY` clause with the desired columns (e.g., (customerID, tenure)) to enable Liquid Clustering on the table.

In [0]:
%sql
CREATE OR REPLACE TABLE batch_inference_liquid_clustering(
  customerID STRING,
  Churn STRING,
  SeniorCitizen DOUBLE,
  tenure DOUBLE,
  MonthlyCharges DOUBLE,
  TotalCharges DOUBLE,
  prediction STRING
  )

In [0]:
%sql
OPTIMIZE batch_inference_liquid_clustering;
ALTER TABLE batch_inference_liquid_clustering
CLUSTER BY (customerID, tenure);


## Clean up Classroom

Run the following cell to remove lessons-specific assets created during this lesson.

In [0]:
DA.cleanup()

Resetting the learning environment:
| dropping the catalog "labuser8891327_1736105957_wp0t_da"...(0 seconds)

Validating the locally installed datasets:
| listing local files...(1 seconds)
| validation completed...(1 seconds total)



## Conclusion

This lab provides you with hands-on experience in batch deployment, covering model inference, Model Registry usage, and the impact of features like Liquid Clustering on performance. you will gain practical insights into deploying models at scale in a batch-oriented environment.


&copy; 2024 Databricks, Inc. All rights reserved.<br/>
Apache, Apache Spark, Spark and the Spark logo are trademarks of the 
<a href="https://www.apache.org/">Apache Software Foundation</a>.<br/>
<br/><a href="https://databricks.com/privacy-policy">Privacy Policy</a> | 
<a href="https://databricks.com/terms-of-use">Terms of Use</a> | 
<a href="https://help.databricks.com/">Support</a>